**Modelo tipo 1 e 2**

* Tipo 1: demanda <= oferta

* Tipo 2: demanda >= oferta

* fim no cliente

In [19]:
import gurobipy as gp
import numpy as np
import pandas as pd
from math import *

In [20]:
df_ori_dest = pd.read_csv('../dados/origem_porto.csv')
df_ori_trans = pd.read_csv('../dados/origem_transbordo.csv')
df_trans_porto = pd.read_csv('../dados/transbordo_porto.csv')

In [21]:
df_supply = pd.read_csv('../dados/supply.csv')
df_cap_port = pd.read_csv('../dados/cap_porto.csv')
df_cap_trans = pd.read_csv('../dados/cap_transbordo.csv')
df_demand = pd.read_csv('../dados/demand.csv')

In [22]:
qnt_orig = df_supply.shape[0]
qnt_trans = df_cap_trans.shape[0]
qnt_port = df_cap_port.shape[0]
qnt_cli = df_demand.shape[0]

In [23]:
df_supply

,0
0,540
1,794
2,853
3,195
4,965
...,...
95,947
96,434
97,776
98,496


In [24]:
df_demand

,0
0,55096
1,22785
2,47568
3,13313
4,51326
...,...
95,54458
96,18341
97,6757
98,22964


In [25]:
df_ori_dest

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,81.429154,65.808838,73.952028,20.188551,82.610217,31.315887,23.499855,45.846352,23.215443,146.203234,...,154.941228,101.135315,92.396502,107.116983,49.615739,20.590949,10.340754,163.937549,66.883062,146.338785
1,72.556075,55.538210,63.689045,29.173138,78.748613,33.674584,20.040279,39.796281,16.800000,140.881794,...,148.570059,96.999728,84.749419,98.956734,39.456467,14.631036,16.677650,158.219614,56.828613,139.691363
2,67.143786,52.321957,60.391893,34.347495,83.114572,39.886238,14.468283,33.550440,10.564847,144.288511,...,151.282686,101.160751,86.646168,100.306857,35.805698,8.504117,22.492381,161.353634,52.812847,142.221371
3,77.463963,64.289147,72.347252,25.197841,87.630497,37.510020,17.736922,40.489801,18.599957,150.665043,...,158.851881,106.076501,95.438625,109.755802,47.730979,15.939887,16.572169,168.209393,64.584729,150.081365
4,68.659391,53.889606,61.966081,32.840171,83.249985,39.011465,14.769482,34.631777,11.559931,144.740584,...,151.925034,101.371081,87.470114,101.254847,37.390737,9.292406,21.150281,161.884897,54.411528,142.908754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,74.838450,56.623953,64.789530,27.380110,76.145377,30.601856,23.122630,42.787363,19.850320,138.633237,...,146.614662,94.464791,83.215998,97.657426,40.852359,17.722483,14.433738,156.079323,58.334210,137.820289
96,74.467238,58.069200,66.213135,27.072170,80.112561,33.307681,19.958357,40.707031,17.626889,142.628491,...,150.552858,98.447858,86.983070,101.300725,41.905646,15.241207,15.037447,160.060549,59.221618,141.731916
97,81.392363,65.867552,74.008778,20.254382,82.835200,31.523655,23.358904,45.735658,23.133698,146.423168,...,155.153230,101.359842,92.590799,107.302757,49.658031,20.504985,10.522357,164.155088,66.913293,146.547860
98,79.257353,62.514766,70.672382,22.272207,79.656864,30.115723,23.395042,44.966832,21.964335,142.939383,...,151.456122,98.132188,88.677682,103.339317,46.496176,19.455632,10.517490,160.590351,63.875880,142.798633


In [26]:
N = [i for i in range(qnt_orig)]
M = [i + qnt_orig + qnt_trans for i in range(qnt_port)]
K = [i + qnt_orig for i in range(qnt_trans)]
O = [i + qnt_orig + qnt_trans + qnt_port for i in range(qnt_cli)]

supply = {}
for i in N:
    supply[i] = df_supply['0'][i]

demand = {}
for i in O:
    demand[i] = df_demand['0'][i - O[0]]

cap_transbordo = {}
for i in K:
    cap_transbordo[i] = df_cap_trans['0'][i - K[0]]

cap_port = {}
for i in M:
    cap_port[i] = df_cap_port['0'][i - M[0]]

cost = {}
for i in N:
    for j in M:
        cost[i, j] = df_ori_dest[str(j - M[0])][i]
    for k in K:
        cost[i, k] = df_ori_trans[str(k - K[0])][i]

for j in M:
    for k in K:
        cost[k, j] = df_trans_porto[str(j - M[0])][k - K[0]]
    for o in O:
        cost[j, o] = 0


In [27]:
oferta_total = 0
for i in supply:
    oferta_total += supply[i]
demanda_total = 0
for i in demand:
    demanda_total += demand[i]

print(oferta_total, demanda_total)

57027 3051285


In [28]:
m = gp.Model("probrema")

In [29]:
X = {}
for i in N:
    for j in M:
        X[i, j] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="X_{}_{}".format(i, j))

for i in N:
    for k in K:
        X[i, k] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="X_{}_{}".format(i, k))
                
for j in M:
    for k in K:
        X[k, j] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="X_{}_{}".format(k, j))

for j in M:
    for o in O:
        X[j, o] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="X_{}_{}".format(j, o))

In [30]:
m.setObjective(
    gp.quicksum((X[i, j] * cost[i, j]) for i in N for j in M) + \
    gp.quicksum((X[i, k] * cost[i, k]) for i in N for k in K) + \
    gp.quicksum((X[k, j] * cost[k, j]) for j in M for k in K),
    sense=gp.GRB.MINIMIZE
)

In [31]:
if oferta_total <= demanda_total:
    for i in N:
        m.addConstr(
            (gp.quicksum(X[i, j] for j in M) + gp.quicksum(X[i, k] for k in K)) == supply[i]
        )
else:
    for i in N:
        m.addConstr(
            (gp.quicksum(X[i, j] for j in M) + gp.quicksum(X[i, k] for k in K)) <= supply[i]
        )

In [32]:
if oferta_total <= demanda_total:
    for o in O:
        m.addConstr(
            gp.quicksum(X[j, o] for j in M) <= demand[o]
        )
else:
    for o in O:
        m.addConstr(
            gp.quicksum(X[j, o] for j in M) == demand[o]
        )

In [33]:
for j in M:
    m.addConstr(
        (gp.quicksum(X[i, j] for i in N) + gp.quicksum(X[k, j] for k in K)) <= cap_port[j]
    )

for k in K:
    m.addConstr(
        gp.quicksum(X[i, k] for i in N) <= cap_transbordo[k]
    )

for k in K:
    m.addConstr(
        gp.quicksum(X[i, k] for i in N) == gp.quicksum(X[k, j] for j in M)
    )

for j in M:
    m.addConstr(
        gp.quicksum(X[j, o] for o in O) == (gp.quicksum(X[i, j] for i in N) + gp.quicksum(X[k, j] for k in K))
    )

In [34]:
rest = m.addConstrs(
    gp.quicksum(X[i, j] for i in N) >= 0 for j in M
)

rest = m.addConstrs(
    gp.quicksum(X[i, k] for i in N) >= 0 for k in K
)

rest = m.addConstrs(
    gp.quicksum(X[k, j] for k in K) >= 0 for j in M
)

In [35]:
# Executa o modelo
m.update()
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 900 rows, 40000 columns and 140000 nonzeros
Model fingerprint: 0xb224407f
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+05]


Presolve removed 300 rows and 0 columns
Presolve time: 0.12s
Presolved: 600 rows, 40000 columns, 110000 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 7.020e+04
 Factor NZ  : 1.016e+05 (roughly 17 MB of memory)
 Factor Ops : 2.372e+07 (less than 1 second per iteration)
 Threads    : 3

Barrier performed 0 iterations in 0.17 seconds (0.05 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Iteration    Objective       Primal Inf.    Dual Inf.      Time
     279    3.1674804e+05   0.000000e+00   0.000000e+00      0s

Solved in 279 iterations and 0.19 seconds (0.06 work units)
Optimal objective  3.167480398e+05


In [36]:
m.write("out.sol")